In [64]:
import math
import pandas as pd
import numpy as np
import random

In [65]:
def distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def get_circle_radius(circle_size):
    return math.sqrt(circle_size/math.pi)

def get_square_half_side(size):
    return math.sqrt(size) / 2

def check_circle_square_collision(circle_x, circle_y, circle_size,
                                square_x, square_y, square_size): 
    
    # Вычисляем радиус круга и половину стороны квадрата
    radius = get_circle_radius(circle_size)
    half_side = get_square_half_side(square_size)
    
    # Находим ближайшую точку квадрата к центру круга
    closest_x = max(square_x - half_side, min(circle_x, square_x + half_side))
    closest_y = max(square_y - half_side, min(circle_y, square_y + half_side))
    
    # Вычисляем расстояние от центра круга до ближайшей точки
    dist = distance(circle_x, circle_y, closest_x, closest_y)
    
    # Если расстояние меньше радиуса, есть коллизия
    return dist < radius

In [66]:
def collision(obj1, obj2):
    if obj1['figure'] == 'circle' and obj2['figure'] == 'circle':
        return (distance(obj1['x'], obj2['x'], obj1['y'], obj2['y']) < get_circle_radius(obj1['size']) + get_circle_radius(obj2['size']))
    
    elif obj1['figure'] == 'rectangle' and obj2['figure'] == 'rectangle':
        return (abs(obj1['x'] - obj2['x']) < (get_square_half_side(obj1['size'])) + get_square_half_side(obj2['size'])) and abs(obj1['y'] - obj2['y']) < (get_square_half_side(obj1['size'])) + get_square_half_side(obj2['size'])
        
    elif (obj1['figure'] in ('rectangle', 'circle') and 
        obj2['figure'] in ('rectangle', 'circle') and 
        obj1['figure'] != obj2['figure']):
        # Определяем, какой объект круг, а какой прямоугольник
        circle_obj = obj1 if obj1['figure'] == 'circle' else obj2
        rect_obj = obj2 if obj1['figure'] == 'circle' else obj1
        
        # Создаем кортежи с нужными данными
        circle = (circle_obj['x'], circle_obj['y'], get_circle_radius(circle_obj['size']))
        square = (rect_obj['x'], rect_obj['y'], rect_obj['size'])
            
        result = check_circle_square_collision(circle[0], circle[1], circle[2],
                                            square[0], square[1], square[2])
        return result

In [67]:
def generate_random_object():
    return {
        'x': random.randint(-5, 5),
        'y': random.randint(-6, 6),
        'size': random.randint(10, 100),
        'figure': random.choice(['circle', 'rectangle']),

        'is_active': random.choice([0, 1]),  
        'color': random.choice(['red', 'blue', 'green', 'yellow']), 
        'priority': random.choice([1, 2, 3, 4, 5]),  
        
        'speed': random.uniform(0, 10),  
        'opacity': random.uniform(0, 1),  
        'rotation': random.randint(0, 359)  
    }

In [68]:
def generate_data(num_rows, num_features=4):
    # Определяем полный набор колонок
    all_columns = [
        'x1', 'y1', 'size1', 'figure1',  
        'is_active1', 'color1', 'priority1', 'speed1', 'opacity1', 'rotation1',  
        'x2', 'y2', 'size2', 'figure2',  
        'is_active2', 'color2', 'priority2', 'speed2', 'opacity2', 'rotation2',  
        'collision'  
    ]
    
    # Выбираем нужное количество признаков (исключая collision)
    if num_features == 4:
        selected_columns = ['x1', 'y1', 'size1', 'figure1', 
                          'x2', 'y2', 'size2', 'figure2', 'collision']
    elif num_features == 7:
        selected_columns = ['x1', 'y1', 'size1', 'figure1', 'is_active1', 'color1', 'priority1',
                          'x2', 'y2', 'size2', 'figure2', 'is_active2', 'color2', 'priority2', 'collision']
    else:  # 10 признаков
        selected_columns = all_columns

    df = pd.DataFrame(columns=selected_columns)
    
    for i in range(num_rows):
        obj1 = generate_random_object()
        obj2 = generate_random_object()
        
        # Создаем словарь с данными для строки
        row_data = {
            'x1': obj1['x'], 'y1': obj1['y'], 'size1': obj1['size'], 'figure1': obj1['figure'],
            'x2': obj2['x'], 'y2': obj2['y'], 'size2': obj2['size'], 'figure2': obj2['figure'],
            'is_active1': obj1['is_active'], 'color1': obj1['color'], 
            'priority1': obj1['priority'], 'speed1': obj1['speed'],
            'opacity1': obj1['opacity'], 'rotation1': obj1['rotation'],
            'is_active2': obj2['is_active'], 'color2': obj2['color'],
            'priority2': obj2['priority'], 'speed2': obj2['speed'],
            'opacity2': obj2['opacity'], 'rotation2': obj2['rotation'],
            'collision': collision(obj1, obj2)
        }
        
        # Фильтруем только нужные колонки
        filtered_row = {k: row_data[k] for k in selected_columns}
        temp_df = pd.DataFrame([filtered_row])
        df = pd.concat([df, temp_df], ignore_index=True)
    
    return df

In [ ]:
datasets = {}

# Параметры размеров
size_ranges = [50, 200, 500, 1000]


# Количество признаков
feature_counts = [4, 7, 10]

# Генерируем датасеты
dataset_counter = 1
for size_range in size_ranges:
    for feature_count in feature_counts:
        datasets[f'dataset_{dataset_counter}'] = generate_data(size_range, feature_count)
        print(f"Dataset {dataset_counter}: {size_range} строк, {feature_count} признаков")
        dataset_counter += 1

# Пример сохранения датасетов в файлы
for name, df in datasets.items():
    df.to_csv(f"{name}.csv", index=False)

Dataset 1: 50 строк, 4 признаков
Dataset 2: 50 строк, 7 признаков
Dataset 3: 50 строк, 10 признаков
Dataset 4: 200 строк, 4 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 5: 200 строк, 7 признаков
Dataset 6: 200 строк, 10 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 7: 500 строк, 4 признаков
Dataset 8: 500 строк, 7 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 9: 500 строк, 10 признаков
Dataset 10: 1000 строк, 4 признаков
Dataset 11: 1000 строк, 7 признаков


C:\Users\user\AppData\Local\Temp\ipykernel_9740\2956248099.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


Dataset 12: 1000 строк, 10 признаков
